In [44]:
class Generate_Database:
    import numpy as np
    import cv2
    from align import AlignDlib
    import dlib
    def __init__(self,model,alignment):
        self.model=model
        self.database={}
        self.alignment=alignment
    
    
    def align_image(self,img,bb):
        alignment=self.alignment
        '''
        OBJECTIVE: Align a face based using DLIB model

        Parameter:
        test - RGB Image
        bb - bounding box around the person to be searched

        Returns:
        Aligned face

        '''

        # return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
        if(bb):
            return alignment.align(96, img, bb, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
        else:
            return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
    
    
    
    
    def generate_encoding(self,img,bb):
        import numpy as np
    
        model=self.model
        '''
        OBJECTIVE: Generates an 128D encoding based on the loaded model

        Parameter:
        test - RGB Image
        bb - bounding box around the person to be searched

        Returns:
        embedded-128D encoding
        '''
        embedded = np.zeros((1, 128)) #Declare a placeholder variable
        #print(m.image_path())    
        # img = load_image(path)
        img = self.align_image(img,bb) #Align the image 
        # plt.imshow(img)
        # scale RGB values to interval [0,1]
        img = (img / 255.).astype(np.float32)
        # obtain embedding vector for image
        embedded= model.predict(np.expand_dims(img, axis=0))[0] #generate encoding
        return embedded
    
    
    def make_database(self,entries):
        database=self.database
       
        '''
        OBJECTIVE: Generate Database

        Parameters:python dictionary with the keys as id and values and image path 
        '''
        
        print('[INFO]:Creating New Database....')    
        for (i,j) in entries.items():
            if i not in database.keys():
                img = entries[i]
                #       print(i)
                database[i] = self.generate_encoding(img,0)
        print('[INFO]:New Database Creation Completed!')
        return database

In [45]:

import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from keras.models import load_model,model_from_json

In [ ]:
from align import AlignDlib
alignment = AlignDlib('shape_predictor_68_face_landmarks.dat')

In [ ]:
def load_final_model():
    '''
    Loads the Final Model saved in the fir 'final_model'
    '''
    # load json and create model
    json_file = open('final_model/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    nn4_small2 = model_from_json(loaded_model_json,custom_objects={'tf': tf})
    # load weights into new model
    nn4_small2.load_weights("final_model/model.h5")
    print("[INFO]:Loaded model from disk")
    return nn4_small2

In [ ]:
model=load_final_model()

In [ ]:
db=Generate_Database(model,alignment)

In [ ]:
import os
import re
path= "./images"
mylist=os.listdir(path)
my_images=[]
myclass=[]

for cl in mylist:
    if re.search(r'.*jpg$',str(cl)) !=None:
        img=cv2.imread(os.path.join(path,cl))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        my_images.append(img)
        myclass.append(os.path.splitext(cl)[0])

        
        
print(my_images[0].shape)
plt.title(myclass[0])
plt.imshow(my_images[0])

In [ ]:
entries={myclass[i]:my_images[i] for i in range(len(myclass))}

In [ ]:
database=db.make_database(entries)

In [ ]:
database["hrishi"].shape